In [17]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import LabelEncoder
import os
import pickle

In [18]:
"""
letter classes = ['B', 'C', 'D', 'F', 'G', 'H', 'J',
           'K', 'L', 'M', 'N', 'P', 'R', 'S', 'T',
            'V', 'W', 'X', 'Y', 'Z']  #these were the classses used, in case label encoder was missing.

number classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']  #these were the classses used, in case label encoder was missing.
"""

label_encoder_numbers_path = 'OCR_models/KNN/label_encoder_numbers.pkl'
label_encoder_letters_path = 'OCR_models/KNN/label_encoder_letters.pkl'

with open(label_encoder_numbers_path, 'rb') as encoder_file:
    label_encoder_numbers = pickle.load(encoder_file)

with open(label_encoder_letters_path, 'rb') as encoder_file:
    label_encoder_letters = pickle.load(encoder_file)

print("LabelEncoders have been loaded successfully.")

letters_number_of_neighbors = 1
numbers_number_of_neighbors = 10

#file paths for KNN models
knn_numbers_model = 'OCR_models/KNN/knn_numbers_model_'+str(numbers_number_of_neighbors)+'.pkl'
knn_letters_model = 'OCR_models/KNN/knn_letters_model_'+str(letters_number_of_neighbors)+'.pkl'

#load KNN Numbers Model
try:
    with open(knn_numbers_model, 'rb') as knn_file:
        knn_numbers = pickle.load(knn_file)
    print(f"KNN Numbers model loaded successfully from {knn_numbers_model}")
except FileNotFoundError:
    print(f"Error: {knn_numbers_model} not found. Please ensure the model is saved correctly.")

#load KNN Letters Model
try:
    with open(knn_letters_model, 'rb') as knn_file:
        knn_letters = pickle.load(knn_file)
    print(f"KNN Letters model loaded successfully from {knn_letters_model}")
except FileNotFoundError:
    print(f"Error: {knn_letters_model} not found. Please ensure the model is saved correctly.")
print(f"({knn_numbers} and {knn_letters}) have been loaded successfully.")


LabelEncoders have been loaded successfully.
KNN Numbers model loaded successfully from OCR_models/KNN/knn_numbers_model_10.pkl
KNN Letters model loaded successfully from OCR_models/KNN/knn_letters_model_1.pkl
(KNeighborsClassifier(n_neighbors=10) and KNeighborsClassifier(n_neighbors=1)) have been loaded successfully.


In [19]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Function to preprocess the image
def preprocess_image(image_path, img_size=20):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Could not load the image at {image_path}")
    img_resized = cv2.resize(img, (img_size, img_size))
    img_flattened = img_resized.flatten().reshape(1, -1)
    return img_flattened

# Function to predict the character of an individual blob image
def predict_plate_digit(image_path, knn_model, label_encoder):
    img_preprocessed = preprocess_image(image_path)
    predicted_label = knn_model.predict(img_preprocessed)
    predicted_character = label_encoder.inverse_transform(predicted_label)
    return predicted_character[0]

# Function to predict the full plate by combining predictions from blob_1 to blob_7
def predict_full_plate(plate_folder_path, knn_model_numbers, knn_model_letters, label_encoder_numbers, label_encoder_letters):
    predicted_plate = ''
    
    for i in range(1, 8):  # assuming 7 blobs (blob_1 to blob_7)
        blob_filename = f'blob_{i}.jpg'
        blob_path = os.path.join(plate_folder_path, blob_filename)
        
        if os.path.exists(blob_path):
            # Predict numbers (first four blobs)
            if i <= 4:
                predicted_char = predict_plate_digit(blob_path, knn_model_numbers, label_encoder_numbers)
            else:  # Predict letters (last three blobs)
                predicted_char = predict_plate_digit(blob_path, knn_model_letters, label_encoder_letters)
            predicted_plate += predicted_char
        else:
            print(f"Warning: {blob_path} does not exist.")
    
    return predicted_plate

# Function to visualize the full plate prediction with the ground truth
def visualize_full_plate(plate_image_path, predicted_plate, actual_plate):
    img = cv2.imread(plate_image_path, cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        raise ValueError(f"Could not load the image at {plate_image_path}")
    
    # Create a directory path including the number of neighbors for both models
    save_dir = f'OCR_results/KNN/numbers_{letters_number_of_neighbors}_letters_{numbers_number_of_neighbors}'
    
    # Ensure the directory exists
    os.makedirs(save_dir, exist_ok=True)
    
    # Construct the save path
    plate_filename = os.path.basename(plate_image_path).replace('.jpg', '').replace('.png', '')  # Remove file extensions
    save_path = os.path.join(save_dir, f'{plate_filename}_prediction.png')
    
    # Display the plate image along with ground truth and predicted plate
    plt.imshow(img, cmap='gray')
    plt.title(f'Ground Truth: {actual_plate}\nPredicted: {predicted_plate}')
    plt.axis('off')
    
    # Save the visualization
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()  # Close the plot to avoid display during batch processing

    print(f"Saved visualization to {save_path}")

# Main function to process and visualize all plates
def process_all_plates(plate_digits_folder, plate_images_folder, knn_model_numbers, knn_model_letters, label_encoder_numbers, label_encoder_letters):
    correct_predictions = 0  # Count of correct characters
    total_characters = 0  # Total number of characters in all plates
    correct_full_plates = 0  # Count of fully correct plates
    total_plates = 0  # Total number of plates processed

    # Traverse the 'frontal' and 'lateral' folders inside 'PlateDigits'
    for plate_type in ['frontal', 'lateral']:
        plate_type_folder = os.path.join(plate_digits_folder, plate_type)
        
        # Iterate through each license plate folder inside 'frontal' or 'lateral'
        for root, dirs, _ in os.walk(plate_type_folder):
            for dir_name in dirs:
                plate_folder_path = os.path.join(root, dir_name)
                
                # The actual plate number is derived from the folder name, removing the '.jpg' extension
                actual_plate = dir_name.replace('.jpg', '')  # Remove '.jpg'

                # Predict the full plate by processing the 7 blobs
                predicted_plate = predict_full_plate(plate_folder_path, knn_model_numbers, knn_model_letters, label_encoder_numbers, label_encoder_letters)

                # Now search in PlateImages based on the 'plate_type' (frontal or lateral)
                plate_image_path_jpg = os.path.join(plate_images_folder, plate_type.capitalize(), f'plate_{actual_plate}.jpg')
                plate_image_path_png = os.path.join(plate_images_folder, plate_type.capitalize(), f'plate_{actual_plate}.png')

                # Visualize the prediction with the plate image if it exists
                if os.path.exists(plate_image_path_jpg):
                    visualize_full_plate(plate_image_path_jpg, predicted_plate, actual_plate)
                    print(f"Processed Plate: {actual_plate} ({plate_type}) | Predicted: {predicted_plate}")
                elif os.path.exists(plate_image_path_png):
                    visualize_full_plate(plate_image_path_png, predicted_plate, actual_plate)
                    print(f"Processed Plate: {actual_plate} ({plate_type}) | Predicted: {predicted_plate}")
                else:
                    print(f"Warning: Plate image for {actual_plate} in {plate_type.capitalize()} does not exist.")

                # Update total number of plates processed
                total_plates += 1
                
                # Check if the entire plate is predicted correctly
                if actual_plate == predicted_plate:
                    correct_full_plates += 1

                # Update total characters and correct predictions (for character-level accuracy)
                total_characters += len(actual_plate)  # Add the number of characters in the actual plate
                correct_predictions += sum(1 for i in range(min(len(actual_plate), len(predicted_plate))) 
                                            if actual_plate[i] == predicted_plate[i])  # Count correct characters

    # Compute and print character-level accuracy
    if total_characters > 0:
        character_accuracy = (correct_predictions / total_characters) * 100
        print(f"\nCharacter-Level Accuracy: {character_accuracy:.2f}% ({correct_predictions} correct out of {total_characters})")
    else:
        print("No characters were processed.")

    # Compute and print plate-level accuracy
    if total_plates > 0:
        plate_accuracy = (correct_full_plates / total_plates) * 100
        print(f"Plate-Level Accuracy: {plate_accuracy:.2f}% ({correct_full_plates} correct plates out of {total_plates})")
    else:
        print("No plates were processed.")


# Load your KNN models and label encoders here
# For example:
# with open('knn_model_numbers.pkl', 'rb') as knn_file:
#     knn_model_numbers = pickle.load(knn_file)
#
# with open('knn_model_letters.pkl', 'rb') as knn_file:
#     knn_model_letters = pickle.load(knn_file)
#
# with open('label_encoder_numbers.pkl', 'rb') as encoder_file:
#     label_encoder_numbers = pickle.load(encoder_file)
#
# with open('label_encoder_letters.pkl', 'rb') as encoder_file:
#     label_encoder_letters = pickle.load(encoder_file)

# Call the function with appropriate paths and models
# process_all_plates('PlateDigits', 'PlateImages', knn_model_numbers, knn_model_letters, label_encoder_numbers, label_encoder_letters)


In [20]:

"""#load the KNN models and label encoders
knn_model_numbers_path = 'knn_numbers_model.pkl'
knn_model_letters_path = 'knn_letters_model.pkl'
label_encoder_numbers_path = 'label_encoder_numbers.pkl'
label_encoder_letters_path = 'label_encoder_letters.pkl'

#load the KNN models
with open(knn_model_numbers_path, 'rb') as knn_file:
    knn_numbers_model = pickle.load(knn_file)

with open(knn_model_letters_path, 'rb') as knn_file:
    knn_letters_model = pickle.load(knn_file)

#load the label encoders
with open(label_encoder_numbers_path, 'rb') as encoder_file:
    label_encoder_numbers = pickle.load(encoder_file)

with open(label_encoder_letters_path, 'rb') as encoder_file:
    label_encoder_letters = pickle.load(encoder_file)"""

print("Models and Label Encoders loaded successfully.")

#set the paths to your dataset
plate_digits_folder = 'PlateDigits'
plate_images_folder = 'PlateImages'

#call the main function to process all plates
process_all_plates(
    plate_digits_folder, 
    plate_images_folder, 
    knn_numbers, 
    knn_letters, 
    label_encoder_numbers, 
    label_encoder_letters
)


Models and Label Encoders loaded successfully.
Saved visualization to OCR_results/KNN/numbers_1_letters_10\plate_1062FNT_prediction.png
Processed Plate: 1062FNT (frontal) | Predicted: 1062FNT
Saved visualization to OCR_results/KNN/numbers_1_letters_10\plate_1565HTS_prediction.png
Processed Plate: 1565HTS (frontal) | Predicted: 1585HTS
Saved visualization to OCR_results/KNN/numbers_1_letters_10\plate_2153GYX_prediction.png
Processed Plate: 2153GYX (frontal) | Predicted: 2159GYX
Saved visualization to OCR_results/KNN/numbers_1_letters_10\plate_2929KXJ_prediction.png
Processed Plate: 2929KXJ (frontal) | Predicted: 2929KXJ
Saved visualization to OCR_results/KNN/numbers_1_letters_10\plate_3340JMF_prediction.png
Processed Plate: 3340JMF (frontal) | Predicted: 9940JNF
Saved visualization to OCR_results/KNN/numbers_1_letters_10\plate_3587DCX_prediction.png
Processed Plate: 3587DCX (frontal) | Predicted: 9687DCX
Saved visualization to OCR_results/KNN/numbers_1_letters_10\plate_4674FHC_predictio